In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr

import fiona
import rasterio
import geopandas

from matplotlib import pyplot as plt

import shapely
import shapely.wkt
import shapely.geometry

import numpy as np
import math
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib
import logging

from prof import Prof
from IPython.core.interactiveshell import InteractiveShell

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)
        
DEBUG=False
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.ERROR)
logging.getLogger('fiona.ogrext').disabled=True # Too noisy
fiona.Env(CPL_DEBUG=DEBUG).__enter__()
rasterio.Env(CPL_DEBUG=DEBUG).__enter__()

# Lecture de l'ENC

In [ ]:
enc_path = 'data/ENC_ROOT/US4MA13M/US4MA13M.000'
m_covr_lyr = fiona.open(enc_path, layer='M_COVR', LIST_AS_STRING=True)

covr_geo = next(f for f in m_covr_lyr if f['properties']['CATCOV'] == 1)['geometry']
repr(covr_geo)[:200]

## Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
covr_path = 'data/covr.geojson'
with fiona.open(covr_path, mode='w', driver='GeoJSON', schema={'properties':{}, 'geometry': 'Polygon'}) as cover_lyr:
    cover_lyr.write({'properties':{}, 'geometry': covr_geo})

# Lecture de la Bathy

## Réduction à la zone de l'ENC

In [ ]:
bathy_ds = gdal.Warp('data/bathy.tif', 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr',
                                            cropToCutline=True,
                                            creationOptions=['COMPRESS=LZW'],
                                            dstNodata=np.nan))
del bathy_ds # ensure release and flush
# On peut faire la même chose avec rasterio.mask.mask

In [ ]:
bathy_ds = rasterio.open('data/bathy.tif')

In [ ]:
assert len(bathy_ds.indexes) == 1
bathy_np = bathy_ds.read()[0]
bathy_ds.nodata

In [ ]:
bathy_np.shape # Northing, Easting

# Lecture de l'ENC (suite)

In [ ]:
soundg_lyr = geopandas.read_file(enc_path, layer='SOUNDG', LIST_AS_STRING=True, SPLIT_MULTIPOINT=True, ADD_SOUNDG_DEPTH=True).to_crs(bathy_ds.crs)
depare_lyr = geopandas.read_file(enc_path, layer='DEPARE', LIST_AS_STRING=True).to_crs(bathy_ds.crs)

In [ ]:
soundg_lyr

# Accès aux pixels

In [ ]:
bathy_ds.xy(0,0), bathy_ds.xy(2000,0), bathy_ds.xy(0, 2000)

In [ ]:
bathy_ds.index(328133.590626, 4722220.06), bathy_ds.index(388133.590626, 4722220.06), bathy_ds.index(328133.590626, 4662220.06)

In [ ]:
bathy_np[bathy_ds.index(415283.0,4717460.3)] # -125,9262

# Replissage des trous

In [ ]:
import rasterio.fill

In [ ]:
# bathy_ds = rasterio.open('data/bathy.tif')

In [ ]:
rasterio.fill.fillnodata(bathy_np, ~np.isnan(bathy_np), smoothing_iterations=100)

In [ ]:
np.nan_to_num(bathy_np, copy=False, nan=10)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(bathy_np)

# Filtration de la Bathy en fonction des profondeurs utilisées dans DEPARE

In [ ]:
depare_lyr

In [ ]:
" ".join(list(depare_lyr.columns))

In [ ]:
depths1=depare_lyr[['DRVAL1', 'DRVAL2']].drop_duplicates().sort_values('DRVAL1')
depths=depths1[(depths1.DRVAL1 != 0) | (depths1.DRVAL2 != 36.5)]
depths

In [ ]:
def write_geotiff(path, data):
    with rasterio.open(
        path,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=bathy_ds.crs,
        transform=bathy_ds.transform,
    ) as dst:
        dst.write(data, 1)

def make_depth_bathy(val1, val2, safety):
    lbathy_np = bathy_np.copy()
#     lbathy_np[np.isnan(lbathy_np)] = 1
    lbathy_np[(lbathy_np >= -val1) | (lbathy_np <= -val2)] = np.nan
#     lbathy_np[(lbathy_np <= -val2+safety)] = np.nan
    plt.figure(figsize = (20,20))
    plt.imshow(lbathy_np)
    write_geotiff(f'dep{val2}.tiff', lbathy_np)
    

In [ ]:
foot=25.4*12/1000
foot

In [ ]:
make_depth_bathy(-2.9, 0.0, safety=1*foot)

In [ ]:
make_depth_bathy(0, 5.4, safety=1*foot)

In [ ]:
make_depth_bathy(5.4, 9.1, safety=1*foot)

In [ ]:
make_depth_bathy(9.1, 18.2, safety=2*foot)

In [ ]:
make_depth_bathy(18.2, 36.5, safety=2*foot)

In [ ]:
make_depth_bathy(36.5, 182.8, safety=3*foot)